In [1]:
import argparse
import os
import logging
import torch
import numpy as np

from tqdm import tqdm
from easydict import EasyDict

from models import build_model
from data import DataManger_Epoch, DataManger_Episode
from logger import setup_logging
from utils import read_config, rmdir, summary, array_interweave, COLOR
from evaluators import recognition_metrics

In [6]:

def log_test(logger_func, attribute_name: list, weight, result_label, result_instance):
    r""" log test from result
    """
    logger_func('instance-based metrics:')
    logger_func('accuracy: %0.4f' % result_instance.accuracy)
    logger_func('precision: %0.4f' % result_instance.precision)
    logger_func('recall: %0.4f' % result_instance.recall)
    logger_func('f1_score: %0.4f' % result_instance.f1_score)

    logger_func('class-based metrics:')
    result = np.stack([result_label.accuracy, result_label.mean_accuracy, result_label.precision, result_label.recall, result_label.f1_score], axis=0)
    result = np.around(result*100, 2)
    result = result.transpose()
    row_format ="{:>20}" + "{:>10}"*6
    
    logger_func(row_format.format('attribute', 'weight', 'accuracy', 'mA', 'precision', 'recall', 'f1_score'))
    
    logger_func(row_format.format(*['-']*7))
    
    for i in range(len(attribute_name)):
        logger_func(row_format.format(attribute_name[i], np.around(weight[i]*100, 2), *result[i].tolist()))

    logger_func(row_format.format(*['-']*7))
    
    logger_func(row_format.format(
        'mean',
        '-',
        round(np.mean(result_label.accuracy)*100, 2),
        round(np.mean(result_label.mean_accuracy)*100, 2),
        round(np.mean(result_label.precision)*100, 2),
        round(np.mean(result_label.recall)*100, 2),
        round(np.mean(result_label.f1_score)*100, 2)))

In [45]:

def compare_class_based(logger_func, attribute_name, weight, result_label1, result_label2, color=COLOR.BOLD):
    
    logger_func('class-based metrics:')
    result1 = np.stack([result_label1.accuracy, result_label1.mean_accuracy, result_label1.precision, result_label1.recall, result_label1.f1_score], axis=0)
    result1 = np.around(result1*100, 2)
    result1 = result1.transpose()

    result2 = np.stack([result_label2.accuracy, result_label2.mean_accuracy, result_label2.precision, result_label2.recall, result_label2.f1_score], axis=0)
    result2 = np.around(result2*100, 2)
    result2 = result2.transpose()

    row_format = "{:>20}{:>12}{:>15}{:>12}{:>20}{:>14}{:>18}"
    logger_func(row_format.format('attribute', 'weight', 'accuracy', 'mA', 'precision', 'recall', 'f1_score'))
    
    logger_func(row_format.format(*['-']*7))

    for i in range(len(attribute_name)):
        row_format = "{:>20}" + "{:>12}"
        for j in range(5):
            if result1[i][j] > result2[i][j]:
                row_format += color + "{:>10}" + COLOR.END +"|{:>5}"
            elif result1[i][j] < result2[i][j]:
                row_format += "{:>10}|" + color + "{:>5}" + COLOR.END
            else:
                row_format += color + "{:>10}" + COLOR.END + "|" + color + "{:>5}" + COLOR.END
        logger_func(row_format.format(attribute_name[i], np.around(weight[i]*100, 2), *array_interweave(result1[i], result2[i]).tolist()))

    row_format = "{:>20}" + "{:>12}" + "{:>10}|{:>5}"*5
    logger_func(row_format.format(*['-']*12))
    
    mean_result1 = np.around(np.mean(np.array([result_label1.accuracy, result_label1.mean_accuracy, result_label1.precision, result_label1.recall, result_label1.f1_score]), axis=1)*100, 2)
    
    mean_result2 = np.around(np.mean(np.array([result_label2.accuracy, result_label2.mean_accuracy, result_label2.precision, result_label2.recall, result_label2.f1_score]), axis=1)*100, 2)
    
    row_format = "{:>20}" + "{:>12}"
    for i in range(5):
        if mean_result1[i] > mean_result2[i]:
            row_format += color + "{:>10}" + COLOR.END +"|{:>5}"
        elif mean_result1[i] < mean_result2[i]:
            row_format += "{:>10}|" + color + "{:>5}" + COLOR.END
        else:
            row_format += color + "{:>10}" + COLOR.END + "|" + color + "{:>5}" + COLOR.END

    logger_func(row_format.format(
        'mean',
        '-',
        round(np.mean(result_label1.accuracy)*100, 2),
        round(np.mean(result_label2.accuracy)*100, 2),
        round(np.mean(result_label1.mean_accuracy)*100, 2),
        round(np.mean(result_label2.mean_accuracy)*100, 2),
        round(np.mean(result_label1.precision)*100, 2),
        round(np.mean(result_label2.precision)*100, 2),
        round(np.mean(result_label1.recall)*100, 2),
        round(np.mean(result_label2.recall)*100, 2),
        round(np.mean(result_label1.f1_score)*100, 2),
        round(np.mean(result_label2.f1_score)*100, 2)))

In [8]:

def test(config, datamanager, logger_func):
    cfg_trainer = config['trainer_colab'] if config['colab'] == True else config['trainer']

    use_gpu = cfg_trainer['n_gpu'] > 0 and torch.cuda.is_available()
    device = torch.device('cuda:0' if use_gpu else 'cpu')
    map_location = "cuda:0" if use_gpu else torch.device('cpu')
    
    model, _ = build_model(config, num_classes=len(datamanager.datasource.get_attribute()))

    logger_func('Loading checkpoint: {} ...'.format(config['resume']))
    checkpoint = torch.load(config['resume'], map_location=map_location)

    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model.to(device)
    
    preds = []
    labels = []

    with tqdm(total=len(datamanager.get_dataloader('test'))) as epoch_pbar:
        with torch.no_grad():
            for batch_idx, (data, _labels) in enumerate(datamanager.get_dataloader('test')):
                data, _labels = data.to(device), _labels.to(device)

                out = model(data)

                _preds = torch.sigmoid(out)
                preds.append(_preds)
                labels.append(_labels)
                epoch_pbar.update(1)
    preds = torch.cat(preds, dim=0)
    labels = torch.cat(labels, dim=0)
    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()

    result_label, result_instance = recognition_metrics(labels, preds)

    return result_label, result_instance

In [2]:
config1 = "config/baseline_peta.yml"
config2 = "config/episode_peta.yml"

resume1 = "/content/drive/Shared drives/REID/HIEN/Models/OSNet_Person_Attribute_Refactor/checkpoints/0731_232453/model_best_accuracy.pth"
resume2 = "/content/drive/Shared drives/REID/HIEN/Models/person_attribute_recognition/checkpoints/0809_231322/model_best_accuracy.pth"

config1 = read_config(config1)
config1.update({'resume': resume1})
config1.update({'colab': True})

config2 = read_config(config2)
config2.update({'resume': resume2})
config2.update({'colab': True})

datamanager1 = DataManger_Epoch(config1['data'])
datamanager2 = DataManger_Episode(config2['data'])

weight = datamanager1.datasource.get_weight('train')

# model1
result_label1, result_instance1 = test(config1, datamanager1, print)

# model 2
result_label2, result_instance2 = test(config2, datamanager2, print)

Downloading...
Downloaded!
Downloading...
Downloaded!


NameError: name 'test' is not defined

In [46]:
compare_class_based(print, datamanager1.datasource.get_attribute(), weight, result_label1, result_label2, COLOR.BLUE)

class-based metrics:
           attribute      weight       accuracy          mA           precision        recall          f1_score
                   -           -              -           -                   -             -                 -
        accessoryHat       10.47      98.2|97.79     93.88|94.64     92.24|86.69     88.55|90.76     90.36|88.68
    accessoryMuffler        8.42     99.13|98.86      96.5| 97.0     96.09|91.73     93.35|94.78      94.7|93.23
    accessoryNothing       74.56     92.67|92.38      87.6|88.04      93.1|93.58     97.53|96.53     95.26|95.04
 accessorySunglasses         3.0     97.91|96.61     70.88|78.44      73.6|43.29      42.2|59.17     53.64| 50.0
            hairLong       23.62      94.5|93.42     92.06| 92.5      89.2| 83.1     87.42|90.74      88.3|86.75
     upperBodyCasual       85.36     94.78|94.43     85.72|86.15     95.53|95.75     98.49|97.83     96.99|96.78
     upperBodyFormal       13.31     95.51|95.16     86.52|87.45     90.12|85

In [7]:
weight = datamanager1.datasource.get_weight('train')
print(weight)

[0.10473684 0.08421053 0.74557894 0.03       0.23621053 0.8535789
 0.13305263 0.07157895 0.04105263 0.02684211 0.14042105 0.01842105
 0.08515789 0.45421052 0.01231579 0.86157894 0.13642105 0.30726317
 0.0348421  0.04631579 0.5121053  0.2983158  0.02157895 0.35663158
 0.21726316 0.19526316 0.19842105 0.29031578 0.2776842  0.078
 0.49189472 0.33578947 0.10210526 0.06147368 0.55157894]


In [36]:
from torch.distributions.multinomial import Multinomial
m = Multinomial(32, torch.exp(1-torch.tensor(weight)))  

In [17]:
attribute_name = datamanager1.datasource.get_attribute()

In [19]:
import random
selected_attribute = random.sample(attribute_name, 8)
print(selected_attribute)

['footwearLeatherShoes', 'lowerBodyShortSkirt', 'carryingOther', 'personalMale', 'footwearShoes', 'upperBodyPlaid', 'upperBodyJacket', 'lowerBodyTrousers']


In [45]:
m.sample()

tensor([0., 0., 1., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [55]:
Multinomial(probs=torch.exp(1-torch.tensor(weight))).rsample(torch.Size([8]))

NotImplementedError: 

In [58]:
torch.multinomial(torch.exp(1-torch.tensor(weight)), 8, replacement=True)

tensor([ 1,  8, 10, 25, 20, 16,  0, 29])

In [72]:
import collections
from collections import defaultdict
result = defaultdict(int)
for _ in range(10000):
    temp = torch.multinomial(torch.exp(1-torch.tensor(weight)), 8, replacement=True)
    for key, value in collections.Counter(temp.numpy()).items():
        result[key] += value
print(result)

TypeError: unsupported operand type(s) for /: 'collections.defaultdict' and 'int'